## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib qt
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('../camera_cal/calibration*.jpg')
# Step through the list and search for chessboard corners
for fname in images:
    img = mpimg.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        #cv2.imshow('img',img)
        #cv2.waitKey(500)

cv2.destroyAllWindows()

## Then write a function cal_undistort to undistort images and plot a sample, saved as distort_vs_undistort.png

In [2]:
def cal_undistort(img, objpoints, imgpoints):
    # Use cv2.calibrateCamera() and cv2.undistort()
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    undist = cv2.undistort(img,mtx,dist)  # Delete this line
    return undist
sampleimg1= mpimg.imread('C:/Users/Yi Zhu/desktop/self-driving_car/CarND-Advanced-Lane-Lines-master/CarND-Advanced-Lane-Lines-master/camera_cal/calibration5.jpg')
undistorted1 = cal_undistort(sampleimg1, objpoints, imgpoints)
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 9))
f.tight_layout()
ax1.imshow(sampleimg1)
ax1.set_title('Original Image', fontsize=30)
ax2.imshow(undistorted1)
ax2.set_title('Undistorted Image', fontsize=30)
plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.)


In [3]:
sampleimg= mpimg.imread('C:/Users/Yi Zhu/desktop/self-driving_car/CarND-Advanced-Lane-Lines-master/CarND-Advanced-Lane-Lines-master/test_images/test3.jpg')
undistorted = cal_undistort(sampleimg, objpoints, imgpoints)
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 9))
f.tight_layout()
ax1.imshow(sampleimg)
ax1.set_title('Original Image', fontsize=30)
ax2.imshow(undistorted)
ax2.set_title('Undistorted Image', fontsize=30)
plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.)

## Then write a function pipeline to show what color and gradient detect repectively, the sample image is saved as  undistorted_vs_pipeline.png 

In [4]:
# Note: img is the undistorted image
def pipeline(img, s_thresh=(170, 255), sx_thresh=(20, 100)):
    img = np.copy(img)
    # Convert to HSV color space and separate the V channel
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    l_channel = hsv[:,:,1]
    s_channel = hsv[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    # Note color_binary[:, :, 0] is all 0s, effectively an all black image. It might
    # be beneficial to replace this channel with something else.
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary))
    return color_binary
    
result = pipeline(undistorted)

# Plot the result
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 9))
f.tight_layout()
ax1.imshow(undistorted)
ax1.set_title('undistorted', fontsize=30)
ax2.imshow(result)
ax2.set_title('Pipeline result', fontsize=30)
plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.)

## Then write a function pipeline_binary to show how color and gradient combine together to produce a binary picture line detection. The sample plot is saved as pipeline_vs_pipelinebinary.png 

In [5]:
# Note: img is the undistorted image
def pipeline_binary(img, s_thresh=(170, 255), sx_thresh=(20, 100)):
    img = np.copy(img)
    # Convert to HSV color space and separate the V channel
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    l_channel = hsv[:,:,1]
    s_channel = hsv[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    # Note color_binary[:, :, 0] is all 0s, effectively an all black image. It might
    # be beneficial to replace this channel with something else.
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1   
    return combined_binary
    
result_binary = pipeline_binary(undistorted)

# Plot the result
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 9))
f.tight_layout()
ax1.imshow(result)
ax1.set_title('pipeline result', fontsize=30)
ax2.imshow(result_binary,cmap='gray')
ax2.set_title('Pipeline_binary result', fontsize=30)
plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.)

## Next, I first  calculated the perspective tranform matrix by using straightline image and corresponding warped image. And write a function to conduct such transform by using the matrix derived above. I provide two sample images. First is the transformation for straightline, saved as straight_vs_birdview.png. And second is the perspective transformed image for test1, saved as test2_birdview.png

In [6]:
straight=mpimg.imread('C:/Users/Yi Zhu/desktop/self-driving_car/CarND-Advanced-Lane-Lines-master/CarND-Advanced-Lane-Lines-master/test_images/straight_lines1.jpg')
src=np.float32([[247,677],[1039,672],[588,453],[692,453]])
dst=np.float32([[270,700],[980,700],[270,0],[980,0]])
img_size=(straight.shape[1],straight.shape[0])
M = cv2.getPerspectiveTransform(src, dst)
Minv = cv2.getPerspectiveTransform(dst, src)
warped_straight= cv2.warpPerspective(straight, M, img_size, flags=cv2.INTER_LINEAR)
warped_test= cv2.warpPerspective(result_binary, M, img_size, flags=cv2.INTER_LINEAR)
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 9))
f.tight_layout()
ax1.imshow(straight)
ax1.set_title('straight_orig', fontsize=30)
ax2.imshow(warped_straight)
ax2.set_title('Pipeline_binary result', fontsize=30)
plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.)

In [7]:
plt.imshow(warped_test,cmap='gray')

## Next let's find the fit line of the image, saved as fittedline.png.


In [8]:
# Take a histogram of the bottom half of the image

#print (warped_test.shape[0]/2)
histogram = np.sum(warped_test[int(warped_test.shape[0]/2):,:], axis=0)
# Create an output image to draw on and  visualize the result
out_img = np.dstack((warped_test, warped_test, warped_test))*255
# Find the peak of the left and right halves of the histogram
# These will be the starting point for the left and right lines
midpoint = np.int(histogram.shape[0]/2)
leftx_base = np.argmax(histogram[:midpoint])
rightx_base = np.argmax(histogram[midpoint:]) + midpoint

# Choose the number of sliding windows
nwindows = 9
# Set height of windows
window_height = np.int(warped_test.shape[0]/nwindows)
# Identify the x and y positions of all nonzero pixels in the image
nonzero = warped_test.nonzero()
nonzeroy = np.array(nonzero[0])
nonzerox = np.array(nonzero[1])
# Current positions to be updated for each window
leftx_current = leftx_base
rightx_current = rightx_base
# Set the width of the windows +/- margin
margin = 100
# Set minimum number of pixels found to recenter window
minpix = 50
# Create empty lists to receive left and right lane pixel indices
left_lane_inds = []
right_lane_inds = []

# Step through the windows one by one
for window in range(nwindows):
    # Identify window boundaries in x and y (and right and left)
    win_y_low = warped_test.shape[0] - (window+1)*window_height
    win_y_high = warped_test.shape[0] - window*window_height
    win_xleft_low = leftx_current - margin
    win_xleft_high = leftx_current + margin
    win_xright_low = rightx_current - margin
    win_xright_high = rightx_current + margin
    # Draw the windows on the visualization image
    #cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
    #cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
    # Identify the nonzero pixels in x and y within the window
    good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
    good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
    # Append these indices to the lists
    left_lane_inds.append(good_left_inds)
    right_lane_inds.append(good_right_inds)
    # If you found > minpix pixels, recenter next window on their mean position
    if len(good_left_inds) > minpix:
        leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
    if len(good_right_inds) > minpix:        
        rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

# Concatenate the arrays of indices
left_lane_inds = np.concatenate(left_lane_inds)
right_lane_inds = np.concatenate(right_lane_inds)

# Extract left and right line pixel positions
leftx = nonzerox[left_lane_inds]
lefty = nonzeroy[left_lane_inds] 
rightx = nonzerox[right_lane_inds]
righty = nonzeroy[right_lane_inds] 

# Fit a second order polynomial to each
left_fit = np.polyfit(lefty, leftx, 2)
right_fit = np.polyfit(righty, rightx, 2)

ploty = np.linspace(0, warped_test.shape[0]-1, warped_test.shape[0] )
left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
plt.imshow(out_img)
plt.plot(left_fitx, ploty, color='yellow')
plt.plot(right_fitx, ploty, color='yellow')
plt.xlim(0, 1280)
plt.ylim(720, 0)

(720, 0)

## write a fit_line function for further use in video detection

In [9]:
def fit_line(warped_test):
    # Take a histogram of the bottom half of the images
    #print (warped_test.shape[0]/2)
    histogram = np.sum(warped_test[int(warped_test.shape[0]/2):,:], axis=0)
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((warped_test, warped_test, warped_test))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(warped_test.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = warped_test.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = warped_test.shape[0] - (window+1)*window_height
        win_y_high = warped_test.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        #cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        #cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)
    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    ploty = np.linspace(0, warped_test.shape[0]-1, warped_test.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    return left_fitx, right_fitx

## Determine the curvature of the lane and vehicle position with respect to center

In [10]:
def calculate_curvature(left_fitx, right_fitx):
    y_eval = np.max(ploty)
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    # Now our radius of curvature is in meters
    return left_curverad, right_curverad 

## print curvature of test_2

In [11]:
    print('left_cur',calculate_curvature(left_fitx, right_fitx)[0], 'm', 'right_cur', calculate_curvature(left_fitx, right_fitx)[1], 'm')


left_cur 584.188387201 m right_cur 621.183276641 m


## print vehicle position respect to center

In [12]:
midpoint=warped_test.shape[1]/2
leftstart=min(left_fitx)
rightstart=max(right_fitx)
lanemiddle=(leftstart+rightstart)/2
offset=abs(midpoint-lanemiddle)*3.7/700
print(offset)
if midpoint-lanemiddle<0:
    pos='left'
else:
    pos='right'

0.609783249922


## draw detected lane back to original image. The lane added picture is saved as   lane_added.png

In [13]:
warp_zero = np.zeros_like(warped_test).astype(np.uint8)
color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

# Recast the x and y points into usable format for cv2.fillPoly()
pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
pts = np.hstack((pts_left, pts_right))

# Draw the lane onto the warped blank image
cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

# Warp the blank back to original image space using inverse perspective matrix (Minv)
newwarp = cv2.warpPerspective(color_warp, Minv, (warped_test.shape[1], warped_test.shape[0])) 
# Combine the result with the original image
result = cv2.addWeighted(undistorted, 1, newwarp, 0.3, 0)
curvature= np.mean(calculate_curvature(left_fitx, right_fitx))
font = cv2.FONT_HERSHEY_SIMPLEX

cv2.putText(result,'Radius of Curvature is '+str(curvature)+'m',(10,50), font, 1,(255,255,255),2)
cv2.putText(result,'Vehicle is '+ str(offset)+'m '+pos +' of the road',(10,80), font, 1,(255,255,255),2)
plt.imshow(result)


## function add_lane(img) works here as the wrapping up of the previous demo of one image and will be applied to vidoe stream.

In [14]:
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.leftx = None
        self.righx = None

In [18]:
from numpy import linalg as LA
def add_lane(img):
    undistorted = cal_undistort(img, objpoints, imgpoints)
    #result = pipeline(undistorted)
    result_binary = pipeline_binary(undistorted)
    warped_test= cv2.warpPerspective(result_binary, M, img_size, flags=cv2.INTER_LINEAR)
    left_fitx, right_fitx = fit_line(warped_test)
    #print(len(trackline.recent_xfitted))
    #print(left_fitx)
    ploty=np.linspace(0, img.shape[0]-1, img.shape[0] )
    #print(max(ploty))
    if len(trackline.recent_xfitted)<n:
        trackline.leftx=left_fitx
        trackline.rightx=right_fitx
        trackline.recent_xfitted.append([left_fitx,right_fitx])
    #else:
        #trackline.recent_xfitted.append([left_fitx,rightfitx])
        #trackline.recent_xfitted.pop(0)
        curvature = calculate_curvature(left_fitx, right_fitx)
        midpoint  =warped_test.shape[1]/2
        leftstart = min(left_fitx)
        rightstart = max(right_fitx)
        lanemiddle = (leftstart+rightstart)/2
        offset=abs(midpoint-lanemiddle)*3.7/700
        #print (leftstart, rightstart,midpoint, lanemiddle,offset)
        if midpoint-lanemiddle<0:
            pos='left'
        else:
            pos='right'
        warp_zero = np.zeros_like(warped_test).astype(np.uint8)
        color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
        # Recast the x and y points into usable format for cv2.fillPoly()
        pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
        pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
        pts = np.hstack((pts_left, pts_right))
        # Draw the lane onto the warped blank image
        cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
        # Warp the blank back to original image space using inverse perspective matrix (Minv)
        newwarp = cv2.warpPerspective(color_warp, Minv, (warped_test.shape[1], warped_test.shape[0])) 
        # Combine the result with the original image
        result = cv2.addWeighted(undistorted, 1, newwarp, 0.3, 0)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(result,'left Radius of Curvature is '+str(curvature[0])+'m',(10,50), font, 1,(255,255,255),2)
        cv2.putText(result,'right Radius of Curvature is '+str(curvature[1])+'m',(10,80), font, 1,(255,255,255),2)
        cv2.putText(result,'Vehicle is '+ str(abs(offset))+'m '+pos +' of the road',(10,110), font, 1,(255,255,255),2)
        #cv2.putText(result,'shape_match'+ str(shapematch),(10,120), font, 1,(255,255,255),2)
        #cv2.putText(result,'line_dist'+ str(rightstart-leftstart),(10,150), font, 1,(255,255,255),2)
        return result
    coef=[]
    coefnorm=[]
    leftxstart_rec=[]
    rightxstart_rec=[]
    for record in trackline.recent_xfitted:
        # Fit new polynomials to x,y in world space
        left_cr = np.polyfit(ploty, record[0], 2)
        right_cr = np.polyfit(ploty, record[1], 2)
        coef.append([left_cr,right_cr])
        coefnorm.append(LA.norm([left_cr,right_cr]))
        leftxstart_rec.append(min(record[0]))
        rightxstart_rec.append(max(record[1]))
    #    leftxstart.append(record[0][warped_test.shape[0]-1])
    #    rightxstart.append(record[1][warped_test.shape[0]-1])
    left_ave_start=trackline.leftx[-1]
    right_ave_start= trackline.rightx[-1]
    trackline.leftx=left_fitx
    trackline.rightx=right_fitx
    average_coef = np.mean(coef, axis=0)
    #print(average_coef)
    trackline.bestfit = average_coef
    trackline.current_fit = coef[-1]
    curvature = calculate_curvature(left_fitx, right_fitx)
    midpoint  =warped_test.shape[1]/2
    leftstart = left_fitx[-1]
    rightstart = right_fitx[-1]
    dist_diff=abs(LA.norm(leftstart-rightstart)-LA.norm(left_ave_start-right_ave_start))
    lanemiddle = (leftstart+rightstart)/2
    offset=abs(midpoint-lanemiddle)*3.7/700
    #print (leftstart, rightstart,offset)
    if midpoint-lanemiddle<0:
        pos='left'
    else:
        pos='right'
    left_fit_cr = np.polyfit(ploty, left_fitx, 2)
    right_fit_cr = np.polyfit(ploty, right_fitx, 2)
    #print(np.array([left_fit_cr,right_fit_cr]))
    trackline.diffs = abs(np.array([left_fit_cr,right_fit_cr])-average_coef)
    #print(LA.norm(trackline.diffs),15*np.std(coefnorm))
    warp_zero_2 = np.zeros_like(warped_test).astype(np.uint8)
    # Recast the x and y points into usable format for cv2.fillPoly()
    #print(ploty)
    pts_left_2 = np.array([np.transpose(np.vstack([trackline.recent_xfitted[-1][0], ploty]))])
    pts_right_2 = np.array([np.flipud(np.transpose(np.vstack([trackline.recent_xfitted[-1][1], ploty])))])
    pts_2 = np.hstack((pts_left_2, pts_right_2))
    # Draw the lane onto the warped blank image
    cv2.fillPoly(warp_zero_2, np.int_([pts_2]), 255)
    warp_zero_1 = np.zeros_like(warped_test).astype(np.uint8)
    pts_left_1 = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right_1 = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts_1 = np.hstack((pts_left_1, pts_right_1))
    # Draw the lane onto the warped blank image
    cv2.fillPoly(warp_zero_1, np.int_([pts_1]), 255)
    ret = cv2.matchShapes(warp_zero_1, warp_zero_2,1,0.0)
    print('ret',ret)
    print ('LA.norm(trackline.diffs)>15*np.std(coefnorm):',LA.norm(trackline.diffs)-15*np.std(coefnorm))
    print('dist_diff',dist_diff)
    if (LA.norm(trackline.diffs)>15*np.std(coefnorm))| (ret>0.001) |(dist_diff>8):
        #print(’unchanged')
        left_fitx = average_coef[0][0]*ploty**2 + average_coef[0][1]*ploty + average_coef[0][2]
        right_fitx = average_coef[1][0]*ploty**2 + average_coef[1][1]*ploty + average_coef[1][2]
       # print(left_fitx)
    else:
        trackline.recent_xfitted.append([left_fitx,right_fitx])
        trackline.recent_xfitted.pop(0)
        trackline.line_base_pos = (midpoint-lanemiddle)*3.7/700
        trackline.radius_of_curvature=curvature
    warp_zero = np.zeros_like(warped_test).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    # Recast the x and y points into usable format for cv2.fillPoly()
    #print(ploty)
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (warped_test.shape[1], warped_test.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(undistorted, 1, newwarp, 0.3, 0)
    font = cv2.FONT_HERSHEY_SIMPLEX
    if midpoint-lanemiddle<0:
        pos='left'
    else:
        pos='right'
    cv2.putText(result,'left Radius of Curvature is '+str(trackline.radius_of_curvature[0])+'m',(10,50), font, 1,(255,255,255),2)
    cv2.putText(result,'right Radius of Curvature is '+str(trackline.radius_of_curvature[1])+'m',(10,80), font, 1,(255,255,255),2)
    cv2.putText(result,'Vehicle is '+ str(abs(trackline.line_base_pos))+'m '+pos +' of the road',(10,110), font, 1,(255,255,255),2)
    #cv2.putText(result,'shape_match'+ str(shapematch),(10,120), font, 1,(255,255,255),2)
    #cv2.putText(result,'line_dist'+ str(rightstart-leftstart),(10,150), font, 1,(255,255,255),2)
    return result

In [19]:
trackline=Line()
n=10

In [20]:
import imageio
imageio.plugins.ffmpeg.download()
from moviepy.editor import VideoFileClip
from IPython.display import HTML
trackline=Line()
white_output = 'C:/Users/Yi Zhu/Desktop/self-driving_car/CarND-Advanced-Lane-Lines-master/CarND-Advanced-Lane-Lines-master/project_video_out_tryy.mp4'
clip1 = VideoFileClip('C:/Users/Yi Zhu/Desktop/self-driving_car/CarND-Advanced-Lane-Lines-master/CarND-Advanced-Lane-Lines-master/project_video.mp4').subclip(37,43)
white_clip = clip1.fl_image(add_lane) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False) 

[MoviePy] >>>> Building video C:/Users/Yi Zhu/Desktop/self-driving_car/CarND-Advanced-Lane-Lines-master/CarND-Advanced-Lane-Lines-master/project_video_out_tryy.mp4
[MoviePy] Writing video C:/Users/Yi Zhu/Desktop/self-driving_car/CarND-Advanced-Lane-Lines-master/CarND-Advanced-Lane-Lines-master/project_video_out_tryy.mp4


  6%|████▉                                                                             | 9/151 [00:06<01:38,  1.44it/s]

ret 0.00014747265914816232
LA.norm(trackline.diffs)>15*np.std(coefnorm): -58.0380995261
dist_diff 0.422108302267


  7%|█████▎                                                                           | 10/151 [00:06<01:38,  1.43it/s]

ret 0.0001385396165145547
LA.norm(trackline.diffs)>15*np.std(coefnorm): -67.8631628197
dist_diff 1.00427350337


...

ret 1.5020886587446824e-05
LA.norm(trackline.diffs)>15*np.std(coefnorm): -92.421738257
dist_diff 6.75068107071


 99%|██████████████████████████████████████████████████████████████████████████████▉ | 149/151 [01:46<00:01,  1.34it/s]

ret 3.955061685728678e-05
LA.norm(trackline.diffs)>15*np.std(coefnorm): -95.4540185501
dist_diff 6.71409121666


 99%|███████████████████████████████████████████████████████████████████████████████▍| 150/151 [01:47<00:00,  1.32it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: C:/Users/Yi Zhu/Desktop/self-driving_car/CarND-Advanced-Lane-Lines-master/CarND-Advanced-Lane-Lines-master/project_video_out_tryy.mp4 

Wall time: 1min 48s


## Try the add_lane method on video stream

In [21]:

import imageio
imageio.plugins.ffmpeg.download()
from moviepy.editor import VideoFileClip
from IPython.display import HTML
trackline=Line()
white_output = 'C:/Users/Yi Zhu/Desktop/self-driving_car/CarND-Advanced-Lane-Lines-master/CarND-Advanced-Lane-Lines-master/project_video_out.mp4'
clip1 = VideoFileClip('C:/Users/Yi Zhu/Desktop/self-driving_car/CarND-Advanced-Lane-Lines-master/CarND-Advanced-Lane-Lines-master/project_video.mp4')

In [22]:
white_clip = clip1.fl_image(add_lane) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False) 

[MoviePy] >>>> Building video C:/Users/Yi Zhu/Desktop/self-driving_car/CarND-Advanced-Lane-Lines-master/CarND-Advanced-Lane-Lines-master/project_video_out.mp4
[MoviePy] Writing video C:/Users/Yi Zhu/Desktop/self-driving_car/CarND-Advanced-Lane-Lines-master/CarND-Advanced-Lane-Lines-master/project_video_out.mp4


  1%|▌                                                                                | 9/1261 [00:06<14:23,  1.45it/s]

ret 2.320864587601612e-05
LA.norm(trackline.diffs)>15*np.std(coefnorm): -191.962638656
dist_diff 3.17369815241


  1%|▋                                                                               | 10/1261 [00:06<14:28,  1.44it/s]

ret 0.00015016572755671698
LA.norm(trackline.diffs)>15*np.std(coefnorm): -209.226638326
dist_diff 6.35166405029


...

ret 7.875671760859682e-07
LA.norm(trackline.diffs)>15*np.std(coefnorm): -379.010805382
dist_diff 4.31961614318


100%|█████████████████████████████████████████████████████████████████████████████▉| 1259/1261 [15:05<00:01,  1.40it/s]

ret 3.56606842936702e-05
LA.norm(trackline.diffs)>15*np.std(coefnorm): -402.003283735
dist_diff 3.92077937971


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [15:06<00:00,  1.41it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: C:/Users/Yi Zhu/Desktop/self-driving_car/CarND-Advanced-Lane-Lines-master/CarND-Advanced-Lane-Lines-master/project_video_out.mp4 

Wall time: 15min 7s
